In [58]:
import pandas as pd
import numpy as np
import dateparser
import datetime
import time

# Helper functions

In [59]:
def remove_trailing_slash(url):
    return url[:-1] if url[-1:] == '/' else url

# Aggregating stats and users



In [60]:
# Load users data
#dfUsers = pd.read_csv("../data/wikia_users.csv", names=['url', 'users_1', 'users_5', 'users_10', 'users_20', 'users_50',
       #' users_100', ' bots'], header=0, skiprows=0)
dfUsers = pd.read_csv("../data/wikia_users.csv", skipinitialspace=True)
dfUsers.drop_duplicates(subset=['url'], inplace=True)
dfUsers.dropna(subset=['url'], inplace=True)
# Remove trailing slash / in case there is at the end of urls
dfUsers['url'] = dfUsers['url'].apply(remove_trailing_slash)
#dfUsers.columns
dfUsers.head()

,url,users_1,users_5,users_10,users_20,users_50,users_100,bots
0,http://spellmagotm.wikia.com,5,1,1,1,1,0,5
1,http://2017-monster-energy-nascar-cup-series.w...,6,3,1,1,1,1,4
2,http://10low46japreligion.wikia.com,7,2,1,1,1,1,5
3,http://de.bibel.wikia.com,4,1,1,1,0,0,2
4,http://indigo-showdown.wikia.com,9,4,3,3,1,0,5


In [61]:
# Load stats data
dfStats = pd.read_csv("../data/20180921-wikia_stats.csv")
dfStats.drop_duplicates(subset=['url'], inplace=True)
dfStats.head()

,creation_date,domain,founding_user_id,headline,hub,id,lang,language,name,stats.activeUsers,...,stats.edits,stats.images,stats.pages,stats.users,stats.videos,title,topic,url,wam_score,stats.nonarticles
0,2012-05-01 13:58:13,spellmagotm.wikia.com,5069110.0,NaN,Games,529058,en,en,Spellmagotm Wiki,0,...,275,11,229,16246020,0,Spellmagotm Wiki,Gaming,http://spellmagotm.wikia.com,0.0,221
1,2017-07-24 22:35:31,2017-monster-energy-nascar-cup-series.wikia.com,32529801.0,NaN,TV,1601247,en,en,2017 Monster Energy NASCAR Cup Series Wiki,0,...,230,40,153,16555972,0,2017 Monster Energy NASCAR Cup Series Wiki,Entertainment,http://2017-monster-energy-nascar-cup-series.w...,0.0,108
2,2009-09-15 23:21:34,10low46japreligion.wikia.com,1602876.0,NaN,Lifestyle,52061,en,en,Ancient Japanese Religion (Daramalan Assignmen...,0,...,470,20,310,16117576,4,Ancient Japanese Religion (Daramalan Assignmen...,Philosophy,http://10low46japreligion.wikia.com,0.0,239
3,2018-08-06 19:13:40,de.bibel.wikia.com,36490113.0,NaN,Books,1782059,de,de,Bibel Wiki,2,...,116,0,94,16592250,0,Bibel Wiki,Entertainment,http://de.bibel.wikia.com,0.0,92
4,2014-05-30 15:43:23,indigo-showdown.wikia.com,25001469.0,NaN,Games,982346,en,en,Indigo showdown Wiki,0,...,242,11,156,16387583,2,Indigo showdown Wiki,Gaming,http://indigo-showdown.wikia.com,0.0,139


In [62]:
dfStats.count()

creation_date        294412
domain               294412
founding_user_id     294377
headline               8267
hub                  294412
id                   294412
lang                 294411
language             294411
name                 294412
stats.activeUsers    294412
stats.admins         294412
stats.articles       294412
stats.discussions    240270
stats.edits          294412
stats.images         294412
stats.pages          294412
stats.users          294412
stats.videos         294412
title                294412
topic                284224
url                  294412
wam_score            294412
stats.nonarticles    294412
dtype: int64

In [63]:
dfStats[['url', 'id']].head()

,url,id
0,http://spellmagotm.wikia.com,529058
1,http://2017-monster-energy-nascar-cup-series.w...,1601247
2,http://10low46japreligion.wikia.com,52061
3,http://de.bibel.wikia.com,1782059
4,http://indigo-showdown.wikia.com,982346


In [64]:
dfIndex = pd.read_csv("../data/20180917-curatedIndex.txt", header=None, names=['url'])
dfIndex.drop_duplicates(inplace=True)

dfIndex.head()

,url
0,http://spellmagotm.wikia.com/
1,http://2017-monster-energy-nascar-cup-series.w...
2,http://10low46japreligion.wikia.com/
3,http://de.bibel.wikia.com/
4,http://indigo-showdown.wikia.com/


In [65]:
# Remove trailing slash / in case there is at the end of urls
dfIndex['url'] = dfIndex['url'].apply(remove_trailing_slash)
dfIndex.head()

,url
0,http://spellmagotm.wikia.com
1,http://2017-monster-energy-nascar-cup-series.w...
2,http://10low46japreligion.wikia.com
3,http://de.bibel.wikia.com
4,http://indigo-showdown.wikia.com


In [66]:
# Merge index and stats in order to identify the wikis without stats according to the Wikia API
mergedStatUserData = pd.merge(dfIndex, dfStats[['url', 'id']], how='left', on='url')
mergedStatUserData.head()

,url,id
0,http://spellmagotm.wikia.com,529058.0
1,http://2017-monster-energy-nascar-cup-series.w...,1601247.0
2,http://10low46japreligion.wikia.com,52061.0
3,http://de.bibel.wikia.com,1782059.0
4,http://indigo-showdown.wikia.com,982346.0


In [67]:
# Merge index, stats and users in order to identify the wikis without stats or without users' information
mergedStatUserData = pd.merge(mergedStatUserData, dfUsers[['url', 'users_1']], how='left', on='url')
mergedStatUserData.head()

,url,id,users_1
0,http://spellmagotm.wikia.com,529058.0,5.0
1,http://2017-monster-energy-nascar-cup-series.w...,1601247.0,6.0
2,http://10low46japreligion.wikia.com,52061.0,7.0
3,http://de.bibel.wikia.com,1782059.0,4.0
4,http://indigo-showdown.wikia.com,982346.0,9.0


In [68]:
print('Wikia Index size: {}'.format(len(dfIndex)))
print('  Wikis with stats: {}'.format(len(mergedStatUserData[~mergedStatUserData['id'].isna()])))
print('  Wikis with number of users: {}'.format(len(mergedStatUserData[~mergedStatUserData['users_1'].isna()])))
print('  Wikis with stats AND number of users: {}'.format(len(mergedStatUserData[(~mergedStatUserData['id'].isna()) & (~mergedStatUserData['users_1'].isna())])))

Wikia Index size: 295660
  Wikis with stats: 294409
  Wikis with number of users: 290602
  Wikis with stats AND number of users: 290465


In [69]:
dfStatsUsers = pd.merge(dfIndex, dfStats, how='left', on=['url'])
dfStatsUsers = pd.merge(dfStatsUsers, dfUsers, how='inner', on=['url'])
dfStatsUsers.dropna(subset=['id'], inplace=True)

dfStatsUsers.head()

,url,creation_date,domain,founding_user_id,headline,hub,id,lang,language,name,...,topic,wam_score,stats.nonarticles,users_1,users_5,users_10,users_20,users_50,users_100,bots
0,http://spellmagotm.wikia.com,2012-05-01 13:58:13,spellmagotm.wikia.com,5069110.0,NaN,Games,529058.0,en,en,Spellmagotm Wiki,...,Gaming,0.0,221.0,5,1,1,1,1,0,5
1,http://2017-monster-energy-nascar-cup-series.w...,2017-07-24 22:35:31,2017-monster-energy-nascar-cup-series.wikia.com,32529801.0,NaN,TV,1601247.0,en,en,2017 Monster Energy NASCAR Cup Series Wiki,...,Entertainment,0.0,108.0,6,3,1,1,1,1,4
2,http://10low46japreligion.wikia.com,2009-09-15 23:21:34,10low46japreligion.wikia.com,1602876.0,NaN,Lifestyle,52061.0,en,en,Ancient Japanese Religion (Daramalan Assignmen...,...,Philosophy,0.0,239.0,7,2,1,1,1,1,5
3,http://de.bibel.wikia.com,2018-08-06 19:13:40,de.bibel.wikia.com,36490113.0,NaN,Books,1782059.0,de,de,Bibel Wiki,...,Entertainment,0.0,92.0,4,1,1,1,0,0,2
4,http://indigo-showdown.wikia.com,2014-05-30 15:43:23,indigo-showdown.wikia.com,25001469.0,NaN,Games,982346.0,en,en,Indigo showdown Wiki,...,Gaming,0.0,139.0,9,4,3,3,1,0,5


In [70]:
# Save to CSV
import time
timestr = time.strftime("%Y%m%d")
dfStatsUsers.to_csv('../data/{}-wikia_stats_users.csv'.format(timestr), index=False)

# Aggregating birthdate

In [90]:
# Load birthdate data
dfBirthDate = pd.read_csv("../data/20180919-wikia_birthdate.csv", names = ['url', 'birthDate'], header=0, usecols=['url', 'birthDate'])
dfBirthDate.drop_duplicates(subset=['url'], inplace=True)
# Remove trailing slash / in case there is at the end of urls
dfBirthDate['url'] = dfBirthDate['url'].apply(remove_trailing_slash)

# Create a new column with the birthdate in datetime format
dfBirthDate['datetime.birthDate'] = pd.to_datetime(dfBirthDate['birthDate'], infer_datetime_format=True, errors='coerce') 
dfBirthDate.head()

,url,birthDate,datetime.birthDate
0,http://spellmagotm.wikia.com,"14:14, May 1, 2012",2012-05-01 14:14:00
1,http://2017-monster-energy-nascar-cup-series.w...,"22:35, July 24, 2017",2017-07-24 22:35:00
2,http://10low46japreligion.wikia.com,"23:21, September 15, 2009",2009-09-15 23:21:00
3,http://de.bibel.wikia.com,"19:14, 6. Aug. 2018",NaT
4,http://indigo-showdown.wikia.com,"15:43, May 30, 2014",2014-05-30 15:43:00


In [91]:
mergedBirthData = pd.merge(dfIndex, dfBirthDate, how='left', on=['url'])
mergedBirthData.head()

,url,birthDate,datetime.birthDate
0,http://spellmagotm.wikia.com,"14:14, May 1, 2012",2012-05-01 14:14:00
1,http://2017-monster-energy-nascar-cup-series.w...,"22:35, July 24, 2017",2017-07-24 22:35:00
2,http://10low46japreligion.wikia.com,"23:21, September 15, 2009",2009-09-15 23:21:00
3,http://de.bibel.wikia.com,"19:14, 6. Aug. 2018",NaT
4,http://indigo-showdown.wikia.com,"15:43, May 30, 2014",2014-05-30 15:43:00


In [92]:
# Merge index, stats, users and birthdate in order to identify the wikis that lack from any kind of information
mergedStatUserBirthData = pd.merge(dfStatsUsers, dfBirthDate, how='left', on=['url'])
mergedStatUserBirthData.head()

,url,creation_date,domain,founding_user_id,headline,hub,id,lang,language,name,...,stats.nonarticles,users_1,users_5,users_10,users_20,users_50,users_100,bots,birthDate,datetime.birthDate
0,http://spellmagotm.wikia.com,2012-05-01 13:58:13,spellmagotm.wikia.com,5069110.0,NaN,Games,529058.0,en,en,Spellmagotm Wiki,...,221.0,5,1,1,1,1,0,5,"14:14, May 1, 2012",2012-05-01 14:14:00
1,http://2017-monster-energy-nascar-cup-series.w...,2017-07-24 22:35:31,2017-monster-energy-nascar-cup-series.wikia.com,32529801.0,NaN,TV,1601247.0,en,en,2017 Monster Energy NASCAR Cup Series Wiki,...,108.0,6,3,1,1,1,1,4,"22:35, July 24, 2017",2017-07-24 22:35:00
2,http://10low46japreligion.wikia.com,2009-09-15 23:21:34,10low46japreligion.wikia.com,1602876.0,NaN,Lifestyle,52061.0,en,en,Ancient Japanese Religion (Daramalan Assignmen...,...,239.0,7,2,1,1,1,1,5,"23:21, September 15, 2009",2009-09-15 23:21:00
3,http://de.bibel.wikia.com,2018-08-06 19:13:40,de.bibel.wikia.com,36490113.0,NaN,Books,1782059.0,de,de,Bibel Wiki,...,92.0,4,1,1,1,0,0,2,"19:14, 6. Aug. 2018",NaT
4,http://indigo-showdown.wikia.com,2014-05-30 15:43:23,indigo-showdown.wikia.com,25001469.0,NaN,Games,982346.0,en,en,Indigo showdown Wiki,...,139.0,9,4,3,3,1,0,5,"15:43, May 30, 2014",2014-05-30 15:43:00


In [93]:
print('Wikia Index size: {}'.format(len(dfIndex)))
print('  Wikis with birthDate: {}'.format(len(mergedBirthData[~mergedBirthData['birthDate'].isna()])))
print('  Wikis with stats AND users AND birthDate: {}'.format(len(mergedStatUserBirthData[(~mergedStatUserBirthData['id'].isna()) & (~mergedStatUserBirthData['birthDate'].isna())])))
print('  Wikis with birthDate in correct datetime format: {}'.format(len(mergedStatUserBirthData[~mergedStatUserBirthData['datetime.birthDate'].isna()])))

Wikia Index size: 295660
  Wikis with birthDate: 294744
  Wikis with stats AND users AND birthDate: 290437
  Wikis with birthDate in correct datetime format: 198973


In [95]:
mergedStatUserBirthData.dropna(subset=['birthDate'], inplace=True)
len(mergedStatUserBirthData)

290437

## New parsing of birthdates

Previous stats show that there are around 100k dates in a non valid datetime format. The reason is that the birthdate string is language dependant so some string were not correctly parsed after loading the birthdate data. We will use the [dateparser module](https://dateparser.readthedocs.io/en/v0.7.0/) and the Wiki language in order to parse the non valid birthdates. 

In [100]:
dates = mergedStatUserBirthData[mergedStatUserBirthData['datetime.birthDate'].isna()]['birthDate'].values
languages = mergedStatUserBirthData[mergedStatUserBirthData['datetime.birthDate'].isna()]['lang'].values

i = 0
newDates = {}
for d in dates:
    try:
        newDates[d] = dateparser.parse(d, languages=[languages[i]])
    except Exception:
        newDates[d] = "NONVALID"
    i+=1

In [101]:
languages

array(['de', 'es', 'de', ..., 'ar', 'it', 'ru'], dtype=object)

In [102]:
dates

array(['19:14, 6. Aug. 2018', '13:01 29 nov 2017', '20:10, 24. Jul. 2012',
       ..., '١٦:٥٤، ديسمبر ١٢، ٢٠١٣', '13:19, dic 18, 2011',
       '14:46, апреля 8, 2010'], dtype=object)

In [103]:
nanBirthDates = mergedStatUserBirthData[mergedStatUserBirthData['datetime.birthDate'].isna()].copy()
noNanBirthDates = mergedStatUserBirthData.dropna(subset=['datetime.birthDate'])

# Add new parsed birthdates
nanBirthDates['datetime.birthDate'] = nanBirthDates['birthDate'].map(newDates)

# Remove the birthdates that remain NA
nanBirthDates.dropna(subset=['datetime.birthDate'], inplace=True)

# Remove NONVALID birthdates 
nanBirthDates = nanBirthDates[~nanBirthDates['datetime.birthDate'].isin(['NONVALID'])].copy()

In [106]:
wikiaDataset = pd.concat([noNanBirthDates,nanBirthDates])
len(wikiaDataset)

277795

In [105]:
wikiaDataset[wikiaDataset['url'] == 'http://de.bibel.wikia.com']

,url,creation_date,domain,founding_user_id,headline,hub,id,lang,language,name,...,stats.nonarticles,users_1,users_5,users_10,users_20,users_50,users_100,bots,birthDate,datetime.birthDate
3,http://de.bibel.wikia.com,2018-08-06 19:13:40,de.bibel.wikia.com,36490113.0,NaN,Books,1782059.0,de,de,Bibel Wiki,...,92.0,4,1,1,1,0,0,2,"19:14, 6. Aug. 2018",2018-08-06 19:14:00


## Thailand calendar

Change date for Thailand calendar: [On 6 September 1940, Prime Minister Phibunsongkhram decreed 1 January 1941 as the start of the year 2484 BE, so year 2483 BE had only nine months. To convert dates from 1 January to 31 March prior to that year, the number to add or subtract is 542; otherwise, it is 543.](https://en.wikipedia.org/wiki/Thai_solar_calendar)

In [ ]:
def changeCalendar(row):
    if row['lang']=='th':
        date = row['datetime.birthDate']
        thYear = date.year
        return date.replace(year = thYear-543)
    else:
        return row['datetime.birthDate'] 
    
wikiaDataset['datetime.birthDate'] = wikiaDataset.apply(changeCalendar, axis=1)

In [ ]:
print('Wikis with stats AND users AND valid birthdates: {}'.format(len(wikiaDataset)))

In [ ]:
# Save to CSV
import time
timestr = time.strftime("%Y%m%d")
wikiaDataset.to_csv('../data/{}-wikia_stats_users_birthdate.csv'.format(timestr), index=False)